In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)    

In [3]:
X = tf.placeholder(tf.float32, shape = [None, 784] )

In [4]:
D_W1 = tf.Variable(xavier_init([784, 128]))
D_B1 = tf.Variable(tf.zeros(shape = [128]))
D_W2 = tf.Variable(xavier_init([128,1]))
D_B2 = tf.Variable(tf.zeros(shape = [1]))

theta_D = [D_W1, D_B1, D_W2, D_B1]

In [5]:
Z = tf.placeholder(tf.float32, shape = [None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_B1 = tf.Variable(tf.zeros(shape = [128]))
G_W2 = tf.Variable(xavier_init([128, 784]))
G_B2 = tf.Variable(tf.zeros(shape = [784]))

theta_G = [G_W1, G_B1, G_W2, G_B2]

In [6]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size = [m, n])

In [7]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_B1)
    D_logit = tf.matmul(D_h1, D_W2) + D_B2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit


In [8]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_B1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_B2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [9]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [10]:
G_sample = generator(Z)
D_real, D_real_logit = discriminator(X)
D_fake, D_fake_logit = discriminator(G_sample)

In [11]:
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real_logit, labels = tf.ones_like(D_real_logit)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logit, labels = tf.zeros_like(D_fake_logit)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logit, labels = tf.ones_like(D_fake_logit)))

In [12]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list = theta_G)

In [13]:
mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
i = 0
for j in range(2000):
    if j % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})        
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
    X_mb, _ = mnist.train.next_batch(mb_size)
    print(X_mb.shape)
    
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict = {X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict = {Z: sample_Z(mb_size, Z_dim)})
    
    if j % 1000 == 0 :
        print('Iteration: {}'.format(j))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()
